In [1]:
d1 = [
    'contributor_count_activity',
    'contributor_count_bot_activity',
    'contributor_count_without_bot_activity',
    'active_C2_contributor_count_activity',
    'active_C1_pr_create_contributor_activity',
    'active_C1_pr_comments_contributor_activity',
    'active_C1_issue_create_contributor_activity',
    'active_C1_issue_comments_contributor_activity',
    'commit_frequency_activity',
    'commit_frequency_bot_activity',
    'commit_frequency_without_bot_activity',
    'org_count_activity',
    'comment_frequency_activity',
    'code_review_count_activity',
    'updated_since_activity',
    'closed_issues_count_activity',
    'updated_issues_count_activity', 
    'recent_releases_count_activity',
    'activity_score_activity'
]

d2 = [
    'contributor_count_codequality',
    'contributor_count_bot_codequality',
    'contributor_count_without_bot_codequality',
    'active_C2_contributor_count_codequality',
    'active_C1_pr_create_contributor_codequality',
    'active_C1_pr_comments_contributor_codequality',
    'commit_frequency_codequality',
    'commit_frequency_bot_codequality',
    'commit_frequency_without_bot_codequality',
    'commit_frequency_inside_codequality',
    'commit_frequency_inside_bot_codequality',
    'commit_frequency_inside_without_bot_codequality',
    'is_maintained_codequality',
    'LOC_frequency_codequality',
    'lines_added_frequency_codequality',
    'lines_removed_frequency_codequality',
    'pr_issue_linked_ratio_codequality',
    'code_review_ratio_codequality',
    'code_merge_ratio_codequality',
    'pr_count_codequality',
    'pr_merged_count_codequality',
    'pr_commit_count_codequality',
    'pr_commit_linked_count_codequality',
    'git_pr_linked_ratio_codequality',
    'code_quality_guarantee_codequality'
]

d3 = [
    'issue_first_reponse_avg_community',
    'issue_first_reponse_mid_community',
    'issue_open_time_avg_community',
    'issue_open_time_mid_community',
    'bug_issue_open_time_avg_community',
    'bug_issue_open_time_mid_community',
    'pr_open_time_avg_community',
    'pr_open_time_mid_community',
    'pr_first_response_time_avg_community',
    'pr_first_response_time_mid_community',
    'comment_frequency_community',
    'code_review_count_community',
    'updated_issues_count_community',
    'closed_prs_count_community',
    'community_support_score_community'
]

d4 = [
    'contributor_count_group_activity',
    'contributor_count_bot_group_activity',
    'contributor_count_without_bot_group_activity',
    'contributor_org_count_group_activity',
    'commit_frequency_group_activity',
    'commit_frequency_bot_group_activity',
    'commit_frequency_without_bot_group_activity',
    'commit_frequency_org_group_activity',
    'commit_frequency_org_percentage_group_activity',
    'commit_frequency_percentage_group_activity',
    'org_count_group_activity',
    'contribution_last_group_activity',
    'organizations_activity_group_activity'
]

custom_feature_parameters = {
    "length": None,
    "large_standard_deviation": [{"r": 0.05}, {"r": 0.1}],
    "mean": None,
    "maximum": None,
    "minimum": None,
    "sum_values": None,
    "variance": None,
    "skewness": None,
    "kurtosis": None,
    "absolute_sum_of_changes": None,
    "mean_abs_change": None,
    "mean_change": None,
    "first_location_of_maximum": None,
    "first_location_of_minimum": None,
    "last_location_of_maximum": None,
    "last_location_of_minimum": None,
}

label_path = "/home/confetti/oss/oss-compass-result/label.csv"
file_path = "/home/confetti/oss/oss-compass-result/segment2/"
remove_path = "/home/confetti/oss/oss-compass-result/lab/remove/"
exist_path = "/home/confetti/oss/oss-compass-result/lab/exist/"
model_list = ['Feature-XGBoost', 'Feature-RandomForest', 'Feature-AdaBoost']

In [2]:
import pandas as pd
import csv
import numpy as np
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
import os

def get_part_feature(LabelPath, FilePath, FeaturePath, labels, cnt):
    LabelDict = {}
    with open(LabelPath, 'r', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            LabelDict[row[1].replace("/", "_") + '.csv'] = row[2]
    del LabelDict['repo.csv']
    df_new = pd.DataFrame()
    for filename in os.listdir(FilePath):
        if filename.endswith('.csv'):
            if LabelDict[filename] != "-1":
                FilePaths = os.path.join(FilePath, filename)
                df = pd.read_csv(FilePaths)
                df.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
                df['name'] = filename
                df = df.loc[:, labels]
                df = df.fillna(0)
                X = extract_features(df, column_id='name', column_sort='grimoire_creation_date',
                                     default_fc_parameters=custom_feature_parameters)
                X = X.reset_index()
                X = X.iloc[:, 1:]
                X['label'] = LabelDict[filename]
                df_new = pd.concat([df_new, X])
    df_new.to_csv(FeaturePath + "features_" + str(cnt) + ".csv")

def featureRead(FeaturePath, cnt):
    featurePath = FeaturePath + "features_" + str(cnt) + ".csv"
    df = pd.read_csv(featurePath)
    X, Y = df.iloc[:, 1:-1], df.iloc[:, -1]
    X[X.columns] = X[X.columns].astype(float)
    impute(X)   # 去除NaN数据
    # features_filtered = select_features(X, Y, fdr_level=1e-6)   # 挑选特征
    # features_filtered.columns为挑选的特征的名字
    # selected = features_filtered.columns
    # selected =  selected.to_numpy()
    # if not os.path.exists("select_features.txt"):
    # np.savetxt("select_features.txt", selected, delimiter=',', fmt='%s')
    # return features_filtered.values, Y.values
    return X.values, Y.values



### 消融实验

In [4]:
remove_lab_1 = d2 + d3 + d4
remove_lab_2 = d1 + d3 + d4
remove_lab_3 = d1 + d2 + d4
remove_lab_4 = d1 + d2 + d3
compare = d1 + d2 + d3 + d4
remove_lab_data = [remove_lab_1, remove_lab_2, remove_lab_3, remove_lab_4, compare]


from utils.evaluation import mix_test_lab
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore")


def remove_lab(data_list, cnt):
    data_list.insert(0, 'grimoire_creation_date')
    data_list.insert(0, "name")
    # get_part_feature(label_path, file_path, remove_path, data_list, cnt)
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    X, Y = featureRead(remove_path, cnt)
    preds, conf_matrix, accuracy, precision, recall, f1, auc = mix_test_lab(model_list, X, Y, kf)
    ans = {
        "preds": preds,
        "conf_matrix": conf_matrix,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc
    }
    return ans


remove_conclusion = []
for i in range(len(remove_lab_data)):
    remove_conclusion.append(remove_lab(remove_lab_data[i], i))

==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================


In [5]:
print(len(remove_conclusion))
for d in remove_conclusion:
    print(d['conf_matrix'], d['accuracy'], d['precision'], d['recall'], d['f1'], d['auc'])

5
[[223  37]
 [ 38 287]] 0.8717948717948718 0.8858024691358025 0.8830769230769231 0.8844375963020031 0.8703846153846154
[[228  32]
 [ 32 293]] 0.8905982905982905 0.9015384615384615 0.9015384615384615 0.9015384615384615 0.8892307692307693
[[227  33]
 [ 38 287]] 0.8786324786324786 0.896875 0.8830769230769231 0.889922480620155 0.8780769230769231
[[224  36]
 [ 36 289]] 0.8769230769230769 0.8892307692307693 0.8892307692307693 0.8892307692307693 0.8753846153846154
[[226  34]
 [ 34 291]] 0.8837606837606837 0.8953846153846153 0.8953846153846153 0.8953846153846153 0.8823076923076922


In [6]:
exist_lab_1 = d1
exist_lab_2 = d2
exist_lab_3 = d3
exist_lab_4 = d4
compare = d1 + d2 + d3 + d4
exist_lab_data = [exist_lab_1, exist_lab_2, exist_lab_3, exist_lab_4, compare]


from utils.evaluation import mix_test_lab
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore")


def exist_lab(data_list, cnt):
    data_list.insert(0, 'grimoire_creation_date')
    data_list.insert(0, "name")
    # get_part_feature(label_path, file_path, exist_path, data_list, cnt)
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    X, Y = featureRead(exist_path, cnt)
    preds, conf_matrix, accuracy, precision, recall, f1, auc = mix_test_lab(model_list, X, Y, kf)
    ans = {
        "preds": preds,
        "conf_matrix": conf_matrix,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc
    }
    return ans


exist_conclusion = []
for i in range(len(exist_lab_data)):
    exist_conclusion.append(exist_lab(exist_lab_data[i], i))

==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================
==================================Feature-XGBoost and Feature-RandomForest and Feature-AdaBoost==================================


In [7]:
print(len(exist_conclusion))
for d in exist_conclusion:
    print(d['conf_matrix'], d['accuracy'], d['precision'], d['recall'], d['f1'], d['auc'])

5
[[225  35]
 [ 31 294]] 0.8871794871794871 0.8936170212765957 0.9046153846153846 0.8990825688073394 0.885
[[226  34]
 [ 34 291]] 0.8837606837606837 0.8953846153846153 0.8953846153846153 0.8953846153846153 0.8823076923076922
[[229  31]
 [ 45 280]] 0.8700854700854701 0.9003215434083601 0.8615384615384616 0.8805031446540882 0.8711538461538462
[[214  46]
 [ 40 285]] 0.852991452991453 0.8610271903323263 0.8769230769230769 0.8689024390243902 0.85
[[231  29]
 [ 40 285]] 0.882051282051282 0.9076433121019108 0.8769230769230769 0.8920187793427231 0.8826923076923077
